In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('../data/example_data.csv')

# Load models

In [3]:
#arquitectura neuronal
class HaloToGalaxyModel(nn.Module):
    def __init__(self, input_size=4, output_size=1, hidden_dim=64):
        super(HaloToGalaxyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_dim) 
        self.fc2 = nn.Linear(hidden_dim, hidden_dim) 
        self.fc3 = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  
        return x

In [4]:
modelfile = 'models/smass_model.pth'
model_smass = HaloToGalaxyModel(5, 3, 100)
model_smass.load_state_dict(torch.load(modelfile, map_location=torch.device('cpu')))
#model_color.load_state_dict(torch.load(modelfile)) if you have gpu

/tmp/ipykernel_22338/4040613244.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_smass.load_state_dict(torch.load(modelfile, map_location=torch.device('cpu')))


<All keys matched successfully>

In [5]:
modelfile = 'models/color-g-i_model.pth'
model_color = HaloToGalaxyModel(5, 3, 100)
model_color.load_state_dict(torch.load(modelfile, map_location=torch.device('cpu')))
#model_color.load_state_dict(torch.load(modelfile)) if you have gpu

/tmp/ipykernel_22338/2452793672.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_color.load_state_dict(torch.load(modelfile, map_location=torch.device('cpu')))


<All keys matched successfully>

In [6]:
modelfile = 'models/smass-color-g-i_model.pth'
model_smass_color = HaloToGalaxyModel(5, 6, 100)
model_smass_color.load_state_dict(torch.load(modelfile, map_location=torch.device('cpu')))
#model_smass_color.load_state_dict(torch.load(modelfile)) if you have gpu

/tmp/ipykernel_22338/658345571.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_smass_color.load_state_dict(torch.load(modelfile, map_location=torch.device('cpu')))

<All keys matched successfully>

# Predictions

In [7]:
input_data = data[['M_h', 'C_h', 'S_h', 'z_h', 'Delta3_h']].to_numpy()
output = data[['smass', 'color']].to_numpy()

In [8]:
input_data

array([[12.48163287,  0.99434969,  0.06208566,  1.27272727, 14.32015055],
       [11.65242996,  1.14203344,  0.09452574,  0.84848485,  1.94253163],
       [12.00267253,  1.00050752,  0.07858547,  0.96969697,  3.37133589]])

In [9]:
input_data = torch.tensor(input_data, dtype=torch.float32)

In [10]:
model_smass.eval()
with torch.no_grad():
    y_pred_smass = model_smass(input_data)

y_pred_smass = np.array(y_pred_smass[:,1]).reshape(-1,1)
#y_pred_smass = np.array(y_pred_smass[:,1].cpu())

In [11]:
model_color.eval()
with torch.no_grad():    
    y_pred_color = model_color(input_data).detach().numpy()
y_pred_color = np.array(y_pred_color[:,1]).reshape(-1,1)
#y_pred_color = np.array(y_pred_color[:,1].cpu())

In [12]:
model_smass_color.eval()
with torch.no_grad():
    y_pred_smass_color = model_smass_color(input_data).detach().numpy()

masa_predicted_join = np.array(y_pred_smass_color[:,1])
color_predicted_join = np.array(y_pred_smass_color[:,4])
#masa_predicted_join = np.array(y_pred_smass_color[:,1].cpu())
#color_predicted_join = np.array(y_pred_smass_color[:,4].cpu())
y_pred_smass_color = np.column_stack((masa_predicted_join, color_predicted_join))

# Show predictions

In [13]:
output

array([[10.78942251,  1.12636185],
       [ 9.91511603,  0.59660339],
       [10.34346282,  0.97583961]])

In [14]:
y_pred_smass

array([[10.910477],
       [ 9.817358],
       [10.319688]], dtype=float32)

In [15]:
y_pred_color

array([[1.1224909],
       [0.6359226],
       [0.7848232]], dtype=float32)

In [16]:
y_pred_smass_color

array([[10.829637  ,  1.1490396 ],
       [ 9.893968  ,  0.61100996],
       [10.311057  ,  0.79153347]], dtype=float32)